# 1. Import and Install Dependencies

In [ ]:
!pip install tensorflow==2.4.1 opencv-python mediapipe sklearn matplotlib

In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# 2. Keypoints using MP Holistic

In [2]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [32]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, #not face mesh
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=1, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=1, circle_radius=2), 
                             mp_drawing.DrawingSpec(color=(0,250,0), thickness=1, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=1, circle_radius=2), 
                             mp_drawing.DrawingSpec(color=(0,250,0), thickness=1, circle_radius=2)
                             ) 

### Test Camera not for collecting images.

In [4]:
cap = cv2.VideoCapture(1)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)

        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

## for testing captured images

In [ ]:
draw_styled_landmarks(frame, results)

plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

In [ ]:
draw_styled_landmarks(frame, results)

plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

# 3. Extract Keypoint Values

In [5]:

# With out Face points  multiple left and hand cordinates to increase accuracy
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, lh, rh, lh, rh, lh, rh, lh, rh, lh, rh, lh, rh, lh, rh, lh, rh, lh, rh, lh, rh, lh, rh]) # magnifiying the hands value for better accuracy

# 4. Setup Folders for Collection

In [6]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data_mx_Handsx10') 

# Actions that we try to detect
# - stands for no sign 

actions = np.array(['---','yego', 'neza', 'bibi','urakoze','isibo','umurenje','igihugu','umujyi wa kigali','kicukiro','nyarugenge'])


In [7]:
# 30 videos worth of data
no_sequences = 50

# Videos are going to be 30 frames in length
sequence_length = 30

# Folder start
start_folder = 1

In [8]:
for action in actions:
    for sequence in range(1,no_sequences+1):
        try:
            os.makedirs(os.path.join(DATA_PATH,action, str(sequence)))
        except:
            pass

# 5. Collect Keypoint Values for Training and Testing

In [13]:
cap = cv2.VideoCapture(1)
# Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.4, min_tracking_confidence=0.4) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(start_folder, start_folder+no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING  COLLECTION', (15,40),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5 , (0,255,0), 1, cv2.LINE_AA)
                    cv2.putText(image,'Collection frames for  {} video number {}'.format(action,sequence),(15,20),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5 , (0,0,255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('RSL Data Collection Feed', image)
                    cv2.waitKey(2000)
                else:       
                    cv2.putText(image,'Collection frames for  {} video number {}'.format(action,sequence),(15,20),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5 , (0,0,255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('RSL Data Collection Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
cap.release()
cv2.destroyAllWindows()

# 6. Preprocess Data and Create Labels and Features

In [9]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [10]:
label_map = {label:num for num, label in enumerate(actions)}

In [11]:
label_map

{'---': 0,
 'yego': 1,
 'neza': 2,
 'bibi': 3,
 'urakoze': 4,
 'isibo': 5,
 'umurenje': 6,
 'igihugu': 7,
 'umujyi wa kigali': 8,
 'kicukiro': 9,
 'nyarugenge': 10}

In [12]:
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [20]:
np.array(sequences).shape

(550, 30, 1518)

In [13]:
np.array(labels).shape

(550,)

In [14]:
X = np.array(sequences)

In [26]:
X.shape

(550, 30, 1518)

In [15]:
y = to_categorical(labels).astype(int)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [17]:
y_test.shape

(110, 11)

# 7. Build and Train LSTM Neural Network

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.optimizers import Adam

In [19]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [20]:
#Free style tunning 

opt = Adam(learning_rate=0.00001)

In [21]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1518)))
model.add(LSTM(64, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [22]:
actions.shape[0]

11

In [23]:
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [33]:
model.fit(X_train, y_train, epochs=200, callbacks=[tb_callback])

Epoch 1/200
14/14 [==============================] - 3s 68ms/step - loss: 2.4003 - categorical_accuracy: 0.1409
Epoch 2/200
14/14 [==============================] - 1s 77ms/step - loss: 2.3938 - categorical_accuracy: 0.1159
Epoch 3/200
14/14 [==============================] - 1s 81ms/step - loss: 2.3885 - categorical_accuracy: 0.0886
Epoch 4/200
14/14 [==============================] - 1s 85ms/step - loss: 2.3824 - categorical_accuracy: 0.0886
Epoch 5/200
14/14 [==============================] - 1s 84ms/step - loss: 2.3761 - categorical_accuracy: 0.0886
Epoch 6/200
14/14 [==============================] - 1s 78ms/step - loss: 2.3673 - categorical_accuracy: 0.0886
Epoch 7/200
14/14 [==============================] - 1s 77ms/step - loss: 2.3551 - categorical_accuracy: 0.0909
Epoch 8/200
14/14 [==============================] - 1s 74ms/step - loss: 2.3316 - categorical_accuracy: 0.1659
Epoch 9/200
14/14 [==============================] - 1s 74ms/step - loss: 2.2873 - categorical_accuracy:

14/14 [==============================] - 1s 87ms/step - loss: 0.0105 - categorical_accuracy: 0.9977
Epoch 146/200
14/14 [==============================] - 1s 88ms/step - loss: 0.0106 - categorical_accuracy: 0.9977
Epoch 147/200
14/14 [==============================] - 1s 84ms/step - loss: 0.0109 - categorical_accuracy: 0.9977
Epoch 148/200
14/14 [==============================] - 1s 82ms/step - loss: 0.0098 - categorical_accuracy: 0.9977
Epoch 149/200
14/14 [==============================] - 1s 78ms/step - loss: 0.0100 - categorical_accuracy: 0.9977
Epoch 150/200
14/14 [==============================] - 1s 81ms/step - loss: 0.0097 - categorical_accuracy: 0.9977
Epoch 151/200
14/14 [==============================] - 1s 82ms/step - loss: 0.0092 - categorical_accuracy: 0.9977
Epoch 152/200
14/14 [==============================] - 1s 83ms/step - loss: 0.0090 - categorical_accuracy: 0.9977
Epoch 153/200
14/14 [==============================] - 1s 84ms/step - loss: 0.0088 - categorical_accur

In [34]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            405248    
                                                                 
 lstm_1 (LSTM)               (None, 30, 64)            33024     
                                                                 
 lstm_2 (LSTM)               (None, 64)                33024     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 11)                363       
                                                                 
Total params: 477,899
Trainable params: 477,899
Non-trai

# 8. Make Predictions

In [24]:
res = model.predict(X_test)

4/4 [==============================] - 1s 38ms/step


In [36]:
actions[np.argmax(res[0])]

'umujyi wa kigali'

In [37]:
actions[np.argmax(y_test[0])]

'umujyi wa kigali'

# 9. Save Weights

In [38]:
model.save('12_Signs_50Videos_30Frames_2_less_layers_Magnified_Hands_x10.h5')

In [ ]:
del model

In [25]:
model.load_weights('12_Signs_50Videos_30Frames_2_less_layers_Magnified_Hands_x10.h5')

# 10. Evaluation using Confusion Matrix and Accuracy

In [26]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [27]:
yhat = model.predict(X_test)

4/4 [==============================] - 0s 34ms/step


In [28]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [29]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[ 94,   0],
        [  0,  16]],

       [[ 99,   0],
        [  0,  11]],

       [[102,   0],
        [  0,   8]],

       [[102,   0],
        [  0,   8]],

       [[ 97,   0],
        [  0,  13]],

       [[101,   0],
        [  0,   9]],

       [[100,   0],
        [  1,   9]],

       [[ 99,   1],
        [  0,  10]],

       [[101,   0],
        [  0,   9]],

       [[103,   0],
        [  0,   7]],

       [[101,   0],
        [  0,   9]]], dtype=int64)

In [35]:
accuracy_score(ytrue, yhat)

1.0

# 11. Test in Real Time

In [35]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.85
init_time = time.time()

cap = cv2.VideoCapture(1)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                       
                    else:
                        sentence.append(actions[np.argmax(res)])
#             else: #clearing
#                 if time.time() - init_time > 15:
#                     print ('its time to clear')
#                     init_time = time.time()
#                     sentence.clear()

            if len(sentence) > 8: 
                sentence = sentence[-8:]

 
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)

        
        # Show to screen
        cv2.imshow('RSL Translation Feed', image)
        
        # clear the sentence gracefully
        if cv2.waitKey(10) & 0xFF == ord('c'):
            sentence.clear()

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

1/1 [==============================] - 0s 31ms/step


1/1 [==============================] - 0s 31ms/step


1/1 [==============================] - 0s 31ms/step


In [33]:
cap.release()
cv2.destroyAllWindows()